# Sentiment Data Files Processing

## Reddit data

These files contain snapshots of the sentiment towards specific coins on reddit. I scraped the top and newest headlines from specific subreddits for each cryptocurrency and then used some Natural Language processing technology to assign it a polarity score from (-1,1) and subjectivity score from(0,1). I have provided the reddit sentiment data in 4 files(BitcoinSent.csv, BitcoincashSent.csv, litecoinSent.csv and EtherSent.csv). 

## Twitter data

This file called cryptoTwitterSent.csv contains a snapshots of the overall crypto sentiment on Twitter. I scraped the tweets from dozens of the most influential crypto twitter accounts and scored them using the same natural language processing technology

## Predicting Price at Next hour Using Machine Learning Models

First we need to import the necessary dependencies, in this demo I will use keras which is a high level neural net library to process signals from the sentiment and price data for Ethereum

In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
import numpy as np
from math import sqrt
from numpy import concatenate
import pandas as pd
from datetime import datetime

Using TensorFlow backend.


Next, we need to load the data. 2hourdelta.csv is a file that contains the prices of Ethereum on gdax joined with the sentiment in cryptoTwitterSent.csv along with the columns oDelta and tDelta, which correspond to the future price change of Ethereum in an hour and in 2 hours respectively. We will be using the price, polarity, and subjectivity as inputs and using the column oDelta as the output to our neural net. 

In [2]:
dffor=pd.read_csv("2hourdelta.csv")
values = dffor['ETH'].values.reshape(-1,1)
polarity = dffor['polarity'].values.reshape(-1,1)
subjectivity = dffor['subjectivity'].values.reshape(-1,1)
delta = dffor['oDelta'].values.reshape(-1,1)
values = values.astype('float32')
polarity = polarity.astype('float32')
subjectivity = polarity.astype('float32')
delta=delta.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
train_size = int(len(scaled) * 0.8)
test_size = len(scaled) - train_size
train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]

create_dataset is a helper function to format data for the neural net

In [3]:
def create_dataset(dataset, look_back, sentiment, subjectivity, d, sent=False, sub=False):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        if i >= look_back:
            a = dataset[i-look_back:i+1, 0]
            a = a.tolist()
            if(sent==True):
                a.append(sentiment[i].tolist()[0])
            if(sub==True):
                a.append(subjectivity[i].tolist()[0])
            dataX.append(a)
            dataY.append(d[i, 0])
    print np.array(dataX)
    return np.array(dataX), np.array(dataY)


In [4]:
look_back=0
trainX, trainY = create_dataset(train, look_back, polarity[0:train_size],subjectivity[0:train_size],delta,True,True)
testX, testY = create_dataset(test, look_back, polarity[train_size:len(scaled)],subjectivity[0:train_size],delta,True,True)

[[0.81661159 0.0542684  0.0542684 ]
 [0.81959707 0.0542684  0.0542684 ]
 [0.81959707 0.06163203 0.06163203]
 ...
 [0.50898832 0.0250487  0.0250487 ]
 [0.50898832 0.0250487  0.0250487 ]
 [0.50900954 0.0250487  0.0250487 ]]
[[0.50898832 0.0250487  0.0542684 ]
 [0.50911623 0.0250487  0.0542684 ]
 [0.50888151 0.0250487  0.06163203]
 ...
 [0.23078156 0.07120238 0.02220455]
 [0.23129332 0.07120238 0.02220455]
 [0.23130399 0.07120238 0.02220455]]


In [5]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

### Looking forward 1 hour with Twitter Sentiment

Now that the data is formatted we will put it in a neural net 

In [6]:
model = Sequential()
model.add(LSTM(100, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(1,activation='relu'))
model.compile(loss='mae', optimizer='adam')
history = model.fit(trainX, trainY, epochs=200, batch_size=512,validation_data=(testX, testY), verbose=1, shuffle=False)

Train on 113044 samples, validate on 28261 samples
Epoch 1/200
113044/113044 [==============================] - 3s 24us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 2/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 3/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 4/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 5/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 6/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 7/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 8/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 9/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1

Epoch 76/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 77/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 78/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 79/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 80/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 81/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 82/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 83/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 84/200
113044/113044 [==============================] - 2s 21us/step - loss: 5.1181 - val_loss: 6.1905
Epoch 85/200
113044

Now let's evaluate our model's performance

In [7]:
yhat = model.predict(testX)
rmse_sent = sqrt(mean_squared_error(yhat.reshape(-1, 1), testY.reshape(-1, 1)))
print('Test RMSE: %.3f' % rmse_sent)

Test RMSE: 8.813


We have a root mean_squared_error of 8.813 which is solid. This model predicts the price of Ethereum in an hour. While the order stays open, it could be helpful to look over a shorter time series....

First lets load the file "1mindelta.csv" that contains the prices of Ethereum on gdax joined with the reddit sentiment for Ethereum(EtherSent.csv) and the column delta the represents the one minute time change of Ethereum we will set this to the output and use the same neural net architecture as above to process this new sentiment data

In [3]:
dfnew=pd.read_csv("1mindelta.csv")
values = dfnew['ETH'].values.reshape(-1,1)
polarity = dfnew['polarity'].values.reshape(-1,1)
subjectivity = dfnew['subjectivity'].values.reshape(-1,1)
delta = dfnew['Delta'].values.reshape(-1,1)

In [11]:
values = values.astype('float32')
polarity = polarity.astype('float32')
subjectivity = polarity.astype('float32')
delta=delta.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
train_size = int(len(scaled) * 0.8)
test_size = len(scaled) - train_size
train, test = scaled[0:train_size,:], scaled[train_size:len(scaled),:]
split = train_size

In [12]:
look_back = 1
trainX, trainY = create_dataset(train, look_back, polarity[0:train_size],subjectivity[0:train_size],delta,True,True)
testX, testY = create_dataset(test, look_back, polarity[train_size:len(scaled)],subjectivity[0:train_size],delta,True,True)

[[0.81661159 0.81959707 0.0542684  0.0542684 ]
 [0.81959707 0.81959707 0.06163203 0.06163203]
 [0.81959707 0.81957573 0.06163203 0.06163203]
 ...
 [0.52346736 0.52363807 0.09012013 0.09012013]
 [0.52363807 0.52504534 0.09012013 0.09012013]
 [0.52504534 0.52505606 0.09012013 0.09012013]]
[[0.52506667 0.52504534 0.09012013 0.0542684 ]
 [0.52504534 0.52506667 0.09012013 0.06163203]
 [0.52506667 0.52487475 0.09012013 0.06163203]
 ...
 [0.22614354 0.22650605 0.10727381 0.02220455]
 [0.22650605 0.22652739 0.10727381 0.02220455]
 [0.22652739 0.22714579 0.10727381 0.02220455]]


In [14]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

### Looking Forward 1 minute with Reddit sentiment

In [16]:
model_short = Sequential()
model_short.add(LSTM(100, input_shape=(trainX.shape[1], trainX.shape[2])))
model_short.add(Dense(1,activation='relu'))
model_short.compile(loss='mae', optimizer='adam')
history = model_short.fit(trainX, trainY, epochs=150, batch_size=512, validation_data=(testX, testY), verbose=1, shuffle=False)

Train on 113137 samples, validate on 28283 samples
Epoch 1/150
113137/113137 [==============================] - 3s 24us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 2/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 3/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 4/150
113137/113137 [==============================] - 2s 22us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 5/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 6/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 7/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 8/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 9/150
113137/113137 [==============================] - 2s 22us/step - loss: 0.4

Epoch 76/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 77/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 78/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 79/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 80/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 81/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 82/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 83/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 84/150
113137/113137 [==============================] - 2s 21us/step - loss: 0.4780 - val_loss: 0.5252
Epoch 85/150
113137

In [17]:
yhat = model_short.predict(testX)

rmse_sent = sqrt(mean_squared_error(yhat.reshape(-1, 1), testY.reshape(-1, 1)))
print('Test RMSE: %.3f' % rmse_sent)


Test RMSE: 1.005


Here the root mean squared error is much lower, which is expected given that its looking over a shorter time series